In [1]:
import arcpy
arcpy.env.overwriteOutput = True 
from arcpy.sa import *
from arcpy import env

import os
import pandas as pd
import numpy as np

workspace = os.path.join("..", "..", 'RISA_yr6_GIT_ignored_files', 'Flo_py_wrkspace5_SLR2ft')
tempspace = os.path.join(workspace, "temp")

outspace = os.path.join(workspace, 'Outputs')
#make sure workspace directory exists
if not os.path.exists(outspace):
    os.makedirs(outspace)
    
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container { width: 95%;} div#menubar-container { width: 85%; } div#maintoolbar-container { width: 99%; } </style> """))

In [2]:
# Find areas where infrastructure is seated below a certain elevation on a DEM   ( Not sure if I need thiss?? # arcpy.CheckOutExtension("Spatial")) 

SLR_level = .0001    # the level of sea level rise to stipulate

# make a 1 or 0 raster of the areas where the raster value is LT or GT the stipliated value (have to pay attention the sign 


#in_raster = os.path.join("..",  'Data/Raw/GIS/3m_DEM', '3m_dem_cp.tif')

in_raster = os.path.join(workspace, 'dep2Wat_SLR2ft.asc')

outraster = os.path.join(tempspace, 'LT1m1mdem.tif')
DaNewraster = Raster(in_raster) < SLR_level  
DaNewraster.save(outraster)

# Convert the raster to polygons
outPolygons = os.path.join(tempspace, 'temp.shp')
arcpy.RasterToPolygon_conversion(outraster, outPolygons, "NO_SIMPLIFY")

## Delete any areas where the raster (now gridcode attribute) is 0
arcpy.MakeFeatureLayer_management(outPolygons, "Temp_Poly")
Selection = arcpy.SelectLayerByAttribute_management("Temp_Poly", "NEW_SELECTION", "gridcode > 0")
arcpy.CopyFeatures_management(Selection, os.path.join(tempspace, 'Flooded_Area.shp'))


#  Clip some infrastructure based on the inundated areas
InfraShpPath = os.path.join("..", 'Data/Raw/GIS/Infrastructure/Clipped')

for shpo in os.listdir(InfraShpPath):
    if shpo.endswith(".shp"):
        intoclip = os.path.join(InfraShpPath, shpo) 
        
        # First copy the original features to calculate and print the total lengths of polylines later
        arcpy.management.CopyFeatures(intoclip, os.path.join(tempspace, "Length_{}".format(shpo)))
        
        # Now do the actual clipping of flooded areas
        out_clipped  = os.path.join(outspace, "Flooded_{}".format(shpo))
        arcpy.Clip_analysis(intoclip, os.path.join(tempspace, 'Flooded_Area.shp'), out_clipped)       
        
        
        
### Calculate some info about the floddedness of the polylines and points
Polyline_infrstructures = ['Relevant_Sewer_Lines_core.shp', 'ASPA_water_Distribution_core.shp', 
                           'ASPA_water_Transmission_core.shp', 'Roads_2006_core.shp'] 

for i in Polyline_infrstructures:
    fc = os.path.join(outspace, "Flooded_"+i)
    arcpy.AddField_management(fc, 'lenFlod_m','DOUBLE')
    arcpy.management.CalculateField(fc, "lenFlod_m", "!SHAPE.geodesicLength@METERS!")   
    field = arcpy.da.TableToNumPyArray (fc, "lenFlod_m", skip_nulls=True)
    flod_len = field["lenFlod_m"].sum()
    print("Length of flooded {} is {} m".format(i, flod_len)) 
    
    fc_OG = os.path.join(tempspace, "Length_{}".format(i))
    arcpy.AddField_management(fc_OG, 'lenOG_m','DOUBLE')
    arcpy.management.CalculateField(fc_OG, "lenOG_m", "!SHAPE.geodesicLength@METERS!")   
    field = arcpy.da.TableToNumPyArray (fc_OG, "lenOG_m", skip_nulls=True)
    tot_len = field["lenOG_m"].sum()
    print("Total Original Length of {} is {} m".format(i, tot_len)) 
    
    pct_flooded = round((flod_len/tot_len)*100, 1)
    print("Percent of flooded {} is {}%  \n".format(i, pct_flooded))
    

# Get some stats on the points 
Point_infrstructures = ['ASPA_CustomerMeters_core.shp', 'ASPA_water_Boosters_core.shp', 'All_sewer_LiftStations_core.shp'] 

for i in Point_infrstructures:
    fc = os.path.join(InfraShpPath, i)
    field = arcpy.da.TableToNumPyArray (fc, "FID", skip_nulls=True)
    tot_things = len(field)
    print("Total number of {} is {}".format(i, tot_things))
    
    floddedones = os.path.join(outspace, "Flooded_{}".format(i))
    field = arcpy.da.TableToNumPyArray (floddedones, "FID", skip_nulls=True)
    flood_things = len(field)
    print("Number of flooded {} is {}".format(i, flood_things))
    
    pct_flooded = round((flood_things/tot_things)*100, 1)
    print("Percent of flooded {} is {}%  \n".format(i, pct_flooded))


Length of flooded Relevant_Sewer_Lines_core.shp is 118.80104820620501 m
Total Original Length of Relevant_Sewer_Lines_core.shp is 42315.5045899074 m
Percent of flooded Relevant_Sewer_Lines_core.shp is 0.3%  

Length of flooded ASPA_water_Distribution_core.shp is 260.865548699304 m
Total Original Length of ASPA_water_Distribution_core.shp is 44057.93550498824 m
Percent of flooded ASPA_water_Distribution_core.shp is 0.6%  

Length of flooded ASPA_water_Transmission_core.shp is 242.06322247121003 m
Total Original Length of ASPA_water_Transmission_core.shp is 47137.500621334475 m
Percent of flooded ASPA_water_Transmission_core.shp is 0.5%  

Length of flooded Roads_2006_core.shp is 288.07781020891997 m
Total Original Length of Roads_2006_core.shp is 76765.56681189116 m
Percent of flooded Roads_2006_core.shp is 0.4%  

Total number of ASPA_CustomerMeters_core.shp is 1329
Number of flooded ASPA_CustomerMeters_core.shp is 1
Percent of flooded ASPA_CustomerMeters_core.shp is 0.1%  

Total numb

### Some context and results notes

TO consider: 
- should report results of surface inundated features (SLR_level of 0) and subsurface inundated features (SLR_level of 1) too


The calibrated base scenario seems to do a reasonably good job of showing where average condition groundwater inundation occurrs. Areas in the present day base scenario where water reaches the ground surface match well with areas where there are canals or other persistant nearshore water features (Figure if need) 


Sewer lines includes all force and gravity mains on the asbuilts which we recognize to be an incomplete dataset, but is the best available provided by the utility. 

In present day conditions, of the 9.11 km total of water mains (44.1 km of distribution, 47.1 km of transmission), if we assume a 1 m depth then a total of 3.4 km are consistently inundated with water = 3.7%
In present day conditions, of the 42.3 km total of sewer lines, if we assume a 1 m depth then a total of 1.4 km are consistently inundated with water = 3.3%  
[STILL NEED BURIED ELECTRICAL LINES] 





In [13]:
1602.151806359655 +1792.070075292258 



3394.221881651913

In [12]:
47137.500621334475 +44057.93550498824 

91195.43612632272

In [14]:
3394.221881651913/91195.43612632272

0.03721920773480683